# RAG相关概念

预训练模型
1. 模型架构设计
    - 根据不同的需求选择合适的算法来实现建模
2. 数据(知识、语料)
    - 互联网公开的数据
3. pre-training
    - 预训练
4. SFT 指令微调
    - 通过大量的高质量数据(人工标注数据)
5. RLHF
    - 模型的安全

### 1. 大模型目前固有的局限性 

 大语言模型(LLM)是概率生成系统  
1. 知识时效性:模型知识截止于训练数据时间点(联网搜索)
2. 推理局限性:本质是概率预测而非逻辑运算，复杂数学推理易出错(DeepSeek-R1的架构有所不同)
3. 专业领域盲区:缺乏垂直领域知识
4. 幻觉现象:可能生成看似合理但实际错误的内容

目前在公司做客服或者问答系统，最好的还是RAG，别上来就微调

如果对于并发或者时效性比较强，那么RAG不合适，因为要检索。最好的是end-to-end端对端模型，中间没有其他过程，模型直接根据提示器输出答案

但是微调也不太好，不一定有效果，达不到95%精度不能上生产

RAG，通过检索的方法增强生成模型的能力，检索是根据nlp中语义相似度来进行检索，将检索出来的内容放到prompt中一起给模型，可以理解为开卷考试。【注】如果检索不到相关的材料，那么response：无法回答你的问题


### 2 RAG工程化

##### 2.1 RAG搭建流程:
1. 文档加载，并按一定条件切割成片段
2. 将切割的文本片段灌入检索引擎
3. 封装检索接口
4. 构建调用流程:Query -> 检索 -> Prompt -> LLM -> 回复

##### 2.2 构建索引：
1. “文档加载”指的是将外部知识库或语料库中的文本数据加载到系统中，以构建一个供检索的数据库。（网页搜索其实也是一种RAG）
2. 切割成片段：（很重要！！！）

    2.1 提升检索效率
    如果文档很长，检索系统在匹配和处理整个文档时会非常耗时，影响响应速度。较小的片段可以更快地被检索系统处理和匹配，加快响应速度。

    2.2 增强相关性
    系统可以更精确地检索出与用户查询最相关的片段，而不是整个文档，从而提供更准确的信息给生成模型。（召回率）【注】召回率一般要达到80%才能投入生产
3. 将切分后的片段做向量化，一个专门的向量模型干这个事
4. 将向量化后的数据存入向量数据库

##### 2.3 检索和生成
将query向量化，比较query和片段的相似度（距离），然后组成prompt，输入给llm，得到response

##### 2.4 环境安装
1. llama-index库，自带RAG pipeline。

    llama-index # pipeline

    llama-index-embeddings-huggingface==0.3.1 # 本地调用 embedding 模型

    llama-index-llms-huggingface==0.3.4 # 调用 llm 做推理

2. 为什么不使用langchain？

    langchain适合多模型集成，多模型切换的场景